In [1]:
import re
import numpy as np
import pandas as pd

def parse_min_max(s):
    if pd.isna(s): return (np.nan, np.nan)
    nums = re.findall(r"[-+]?\d*\.\d+|\d+", str(s))
    if len(nums) >= 2:
        return float(nums[0]), float(nums[1])
    return (np.nan, np.nan)

def parse_working_value(s):
    if pd.isna(s): return np.nan
    m = re.search(r"([-+]?\d*\.\d+|\d+)", str(s).replace(",", "."))
    return float(m.group(0)) if m else np.nan

def compute_hi_for_row(row, warn_margin=0.1):
    val = parse_working_value(row.get("WORKING_VALUE_ONBOARD", np.nan))
    low, high = parse_min_max(row.get("MIN_MAX_THRESHOLDS", ""))
    if np.isnan(val) or np.isnan(low) or np.isnan(high):
        return np.nan, "Unknown"
    if low <= val <= high:
        return 1.0, "Healthy"
    elif (low*(1-warn_margin)) <= val <= (high*(1+warn_margin)):
        return 0.5, "Warning"
    else:
        return 0.0, "Critical"

def evaluate_dataframe(df):
    df = df.copy()
    df["Health_Index"], df["Predicted_Status"] = zip(*df.apply(compute_hi_for_row, axis=1))
    return df


C:\Users\CSE\anaconda3\envs\lexenv\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\CSE\anaconda3\envs\lexenv\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\CSE\anaconda3\envs\lexenv\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
